## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

### Imports

In [1]:
# Libraries
import cv2
import glob
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pickle
import sklearn

from ipywidgets import interact, interactive, fixed
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Other
%matplotlib inline

### Paths

In [2]:
# Camera Calibration
path_calImages = 'camera_cal'

# Images
path_imgIn = 'test_images/'
path_imgOut = 'output_images/'

# Videos
path_vidIn = 'test_videos/'
path_vidOut = 'output_images/'

### Parameters

In [3]:
# Source and destination points
src = np.float32([[595, 450], # top left
                  [689, 450], # top right
                  [216, 719], # bottom left
                  [1111, 719]]) # bottom right
dst = np.float32([[380, 0], # top left
                  [900, 0], # top right
                  [380, 720], # bottom left
                  [900, 720]]) # bottom right

# Pixel distance (meters)


### Classes

In [4]:
# Frame class stores image processing features
class Frame():
    def __init__(self):
        # Image Depictions
        self.orig = None # raw image
        self.undst = None # camera calibrated original image
        self.warp = None # aerial view of undistorted image
        self.gray = None # grayscale of warped image
        self.color_binary = None # binary grayscale using color space thresholds
        self.sobel_binary = None # binary grayscale using sobel thresholds
        self.combined_binary = None # binary combination of color and sobel binaries
        self.detected_lanes = None # colormap of detected lanes
        self.final = None # depiction of lane mask on undistorted image
        self.collage = None # collage of all image steps
        
        # Image Properties
        self.M = None
        self.Minv = None
        self.objpts = None
        self.imgpts = None
        
        # Lane Lines
        self.lineLeft = Line()
        self.lineRight = Line()
        
        # Lane Properties
        self.laneRad = None
        self.lanePos = None
        
# Line class stores line calculation features
class Line():
    def __init__(self):
        # Track if line characteristics were already found
        self.found = False
        
        # Track line polynomials
        self.polyNew = [] # new polynomial fit
        self.polyOld = [] # existing polynomial fits
        self.polyWorldNew = []
        self.polyDiff = np.array([0, 0, 0], dtype='float')
        
        # Track line indeces
        self.idx_X = None
        self.idx_Y = None
        
        # Line curvature
        self.curveRad = None
        
    def isFound(self):
        return self.found
    
    def update(self, polyNew, polyTop, idx_X, idx_Y):
        
        if self.found:
            self_polyDiff = self.polyNew[-1] - polyNew
            
        # Handler for new polynomial
        if polyNew is not None:
            self.found = True
            self.idx_X = idx_X
            self.idx_Y = idx_Y
            
            # Update polynomials
            self.poly
            
            
        

### Helper Functions

In [5]:
def undistort(Frame):
    '''
    undistort image using camera calibration points
    input: raw image, objpoints, imgpoints
    output: undistorted image
    '''
    # Gather necessary Frame properties
    img = Frame.orig
    objpts = Frame.objpts
    imgpts = Frame.imgpts
    img_size = (img.shape[1], img.shape[0])
    # Camera calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, img_size, None,None)
    undst = cv2.undistort(img, mtx, dist, None, mtx)
    Frame.undst = undst
    
def unwarp(Frame, src, dst):
    '''
    unwarps image using source and destination points
    input: undistorted image
    output: unwarped image, Minv
    '''
    # Gather necessary Frame properties
    undst = np.copy(Frame.undst)
    img_size = (undst.shape[1], undst.shape[0])
    # Store warp borders
    srcBorder = np.array([[src[0][0],src[0][1]],[src[1][0],src[1][1]], [src[3][0],src[3][1]],[src[2][0],src[2][1]]], np.int32)
    Frame.srcBorder = srcBorder
    dstBorder = np.array([[dst[0][0],dst[0][1]],[dst[1][0],dst[1][1]], [dst[3][0],src[3][1]],[dst[2][0],dst[2][1]]], np.int32)
    Frame.dstBorder = dstBorder
    # Get and store transform parameters
    M = cv2.getPerspectiveTransform(src, dst)
    Frame.M = M
    Minv = cv2.getPerspectiveTransform(dst, src)
    Frame.Min = Minv
    # Do the warp
    warp = cv2.warpPerspective(undst, M, img_size, flags=cv2.INTER_LINEAR)
    Frame.warp = warp
    
def rectify(Frame):
    '''
    utilizes the pipeline outlined above: (HLS[2] & LAB[0]) | (Sobel_abs & Sobel_mag & Sobel_dir)
    '''
    # Gather necessary Frame properties
    warp = cv2.cvtColor(Frame.warp, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY)
    # Set color binary threshold parameters
    colorThres_low = 150
    colorThres_high = 255
    # Set sobel binary threshold parameters
    linKernel = 3
    linThres_min = 15
    linThres_max = 255
    radKernel = 1
    radThres_min = 0.01
    radThres_max = 0.4
    
    # Color
    s_channel = cv2.cvtColor(warp, cv2.COLOR_RGB2HLS)[:,:,2] # HLS, Schannel
    l_channel = cv2.cvtColor(warp, cv2.COLOR_RGB2Lab)[:,:,0] # LAB, L channel

    # Sobel
    sobel_abs = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0)) # x direction
    sobel_abs = np.uint8(255 * sobel_abs / np.max(sobel_abs)) # equalize to [0,255]
    sobel_magx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=linKernel) # x direction
    sobel_magy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=linKernel) # y direction
    sobel_mag = np.sqrt(sobel_magx**2 + sobel_magy**2) # normalize across x and y
    sobel_mag = np.uint8(255 * sobel_mag / np.max(sobel_mag)) # equalize to [0,255]
    sobel_dirx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=radKernel)
    sobel_diry = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=radKernel)
    sobel_dir = np.arctan2(np.absolute(sobel_diry), np.absolute(sobel_dirx))

    # Combined Binaries
    color_binary = np.zeros_like(s_channel)
    color_binary[(s_channel > colorThres_low) & (l_channel > colorThres_low)] = 1
    Frame.color_binary = color_binary
    sobel_binary = np.zeros_like(sobel_abs)
    sobel_binary[((sobel_abs > linThres_min) & (sobel_abs <= linThres_max)) &
                 ((sobel_mag > linThres_min) & (sobel_mag <= linThres_max)) &
                 ((sobel_dir > radThres_min) & (sobel_dir <= radThres_max))] = 1
    Frame.sobel_binary = sobel_binary
    combined_binary = np.zeros_like(color_binary)
    combined_binary[(color_binary == 1) | (sobel_binary == 1)] = 1
    Frame.combined_binary = combined_binary
    
def lineFinder(Frame):
    print('...')
    
def lineRefiner(Frame):
    print('...')
    
def calcPos(Frame):
    print('...')
    
    
def drawLane(Frame):
    print('...')
    
    
def drawLine(img, poly, color):
    print('...')
    
    
def drawText(img, Frame):
    print('...')
    
    
def drawFinal(Frame):
    print('...')
    
    
def visualize(name, img):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, img)
    cv2.waitKey(1)
    
def visualizeAll(Frame):
    print('...')
    
    
def processImage(img):
    Frame.orig = img
    undistort(Frame)
    unwarp(Frame)
    rectify(Frame)
    '''if (Frame.leftLine.isFound() and Frame.rightLine.isFound()):
        lineRefiner(Frame)
    else:
        lineFinder(Frame)
    drawLane(Frame)
    drawFinal(Frame)
    collage(Frame)
    visualize('collage', Frame.collage)'''
    visualize('rectified', Frame.combined_binary)
    

### Initialization

In [6]:
count = 0

In [7]:
# Global parameter
if count == 0:
    Frame = Frame()
    count += 1

# Generate object points
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

#Arrays to store object points and image points from all the images
Frame.objpoints = [] #3d points in real world space
Frame.imgpoints = [] # 2d points in image plane

#Make a list of calibration images
images = glob.glob(path_calImages + '*.jpg')

#Step through the lsit and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #find corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    #If found, add object points, image points
    if ret == True:
        Frame.objpoints.append(objp)
        Frame.imgpoints.append(corners)

        #Draw and display the corners

        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(1)

#cv2.destroyAllWindows()

### Test on Image

In [8]:
test_images = glob.glob(path_imgIn + '*.jpg')
n_test_images = int(len(test_images))

test_image = cv2.imread(test_images[np.random.randint(n_test_images)])
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

processed_image = processImage(test_image)

error: /Users/jenkins/miniconda/1/x64/conda-bld/conda_1486587097465/work/opencv-3.1.0/modules/calib3d/src/calibration.cpp:3314: error: (-215) nimages > 0 in function calibrateCamera


### Test on Video